<a href="https://colab.research.google.com/github/dani-lbnl/mudit/blob/main/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### import packages

In [1]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-compute-11-1
  cuda-nsight-systems-10-1 cuda-nsight-systems-

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
import gensim
from bs4 import BeautifulSoup
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import numpy as np

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
def get_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument("--disable-infobars")
  # open it, go to a website, and get results
  wd = webdriver.Chrome('chromedriver',options=options)
  return wd

### import and process data from pubs.xlsx

In [5]:
def preprocess_lda(data):
  stop_words = stopwords.words('english')

  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  data_words_nostops = remove_stopwords(data)

  # Create Dictionary 
  id2word = corpora.Dictionary(data_words_nostops)  
  # Create Corpus 
  texts = data_words_nostops  
  # Term Document Frequency 
  corpus = [id2word.doc2bow(text) for text in texts]  

  words = []
  for word in texts:
    words.extend(word)

  return corpus, id2word, words

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
df = pd.read_table('/content/drive/MyDrive/LBNL/Pubs.xls')
abstracts = list(df[~df['Abstract'].isna()]['Abstract'])

corpus_abs, id2word_abs, words_abs = preprocess_lda(abstracts)

### Word2Vec model for something

In [ ]:
from gensim.models import KeyedVectors
# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('drive/MyDrive/LBNL/GoogleNews-vectors-negative300.bin', binary=True)
# Access vectors for specific words with a keyed lookup:
vector = model['easy']
vector.shape

### using similarity for something

In [13]:
def get_sim(num_topics):
  lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_abs,
                                            id2word=id2word_abs,
                                            num_topics=num_topics, 
                                            random_state=100,
                                            update_every=1,
                                            #  chunksize=100,
                                            passes=30,
                                            alpha='auto',
                                            per_word_topics=True)

  similarity_list = []

  for i in range(num_topics):
    terms_1 = [id2word_abs[term[0]] for term in lda_model.get_topic_terms(i)]
    for j in range(i,num_topics):
      terms_2 = [id2word_abs[term[0]] for term in lda_model.get_topic_terms(j)]
      cs = 0
      count = 0
      # print(terms_1, terms_2)
      for val1 in terms_1:
        for val2 in terms_2:
          # print(val1,val2)
          if(val1 in model and val2 in model):
            count += 1
            cs += cosine_similarity(model[[val1]],model[[val2]])[0][0]
      cs = cs/count
      similarity_list.append(cs)
      # print(count)

  return np.array(similarity_list).mean()

x = range(5,50,5)
topic_sim_num = []

for i in x:
  a = get_sim(i)
  print(a)
  topic_sim_num.append(a)

NameError: ignored